In [ ]:
import numpy as np
import sys
import os

In [ ]:
def extract_freq(file,atoms):
    """
    This function extract vibrational frequencies and vibrational coordinates
    from a geometry optimization log file. returns a list of vibrational
    frequencies, freq_list and a list of vibrational coordinate matrices, mat_list
    """
    freq_list=[]            #list of vibrational frequencies
    mat_list=[]             #list of vibrational coordinate matrices
    
    #Open a geometry optimization log file and extract vibrational parameters
    with open(file, 'r') as file: 
        line = file.readline()
        while line:

            if 'Frequencies' in line :
                freq1=0
                freq2=0
                freq3=0
                mat1=np.zeros((atoms,3))
                mat2=np.zeros((atoms,3))
                mat3=np.zeros((atoms,3))

                data = line.split()
                freq1 = np.float(data[2])
                freq_list.append(freq1)
                freq2 = np.float(data[3])
                freq_list.append(freq2)
                freq3 = np.float(data[4])
                freq_list.append(freq3)

                for i in range(4):
                    line=file.readline()
                for i in range(atoms):
                    data2 = file.readline().split()
                    data2 = np.array(data2, dtype=float)
                    mat1[i,:] = data2[2:5]
                    mat2[i,:] = data2[5:8]
                    mat3[i,:] = data2[8:11]

                mat_list.append(mat1)
                mat_list.append(mat2)
                mat_list.append(mat3)

            line = file.readline()
        return freq_list, mat_list



In [ ]:
def extract_initial_coordinates(file,atoms):
    """ This function extracts the initial coordinates from the log file.
    Returns a matrix of initial coordinates, coordinates,
    and a list of atomic numbers, atom_list.
    """
    atom_list=np.zeros(atoms,dtype=int)
    with open(file, 'r') as file:
        line = file.readline()
        while line:
            if 'Coordinates' in line:
                coordinates=np.zeros((atoms,3))
                for i in range(2):
                    line=file.readline()


                for i in range(atoms):
                    data = file.readline().split()
                    data = np.array(data, dtype=float)
                    coordinates[i,:] = data[3:6]
                    atom=int(data[1])
                    atom_list[i]=atom
            line = file.readline()
    return coordinates, atom_list


In [ ]:
def new_coordinates(initial_coord,vib1,vib2,stepsize,steps,atoms):
    """ This function generates new coordinates by moving all atoms
    along two vibrational coordinates, vib1 and vib2, in steps
    of stepsize. Returns a list of new coordinate sets
    """
    coordinates_mat=np.zeros((2*steps-1,2*steps-1),dtype=object)
    for i in range(steps):
        for j in range(steps):
            coordinates=np.zeros((atoms,3))
            coordinates=initial_coord+stepsize*(i*vib1+j*vib2)
            coordinates_mat[steps-1+j,steps-1+i]=coordinates

    for i in range(steps):
        for j in range(steps):
            coordinates=np.zeros((atoms,3))
            coordinates=initial_coord+stepsize*(-i*vib1+j*vib2)
            coordinates_mat[steps-1+j,steps-1-i]=coordinates

    for i in range(steps):
        for j in range(steps):
            coordinates=np.zeros((atoms,3))
            coordinates=initial_coord+stepsize*(i*vib1-j*vib2)
            coordinates_mat[steps-1-j,steps-1+i]=coordinates

    for i in range(steps):
        for j in range(steps):
            coordinates=np.zeros((atoms,3))
            coordinates=initial_coord+stepsize*(-i*vib1-j*vib2)
            coordinates_mat[steps-1-j,steps-1-i]=coordinates
    return coordinates_mat


In [ ]:
def input_files(new_coordinates,atom_list,basename,header_file,steps):
    """
    This function generates input files  given the new coordinates list, 
    the atom list, the basename and a header_file containing the link0 
    commands, route section, title card, multiplicity and charge.
    """
    for i in range(2*steps-1):
        for j in range(2*steps-1):

            coordinates=new_coordinates[i,j]
            with open(basename+str(i)+'_'+str(j)+'.com', 'w') as file:
                with open(header_file,'r') as header:
                    for line in header:
                        file.write(line.replace("name.chk",basename+str(i)+'_'+str(j)+'.chk'))
                for j in range(len(atom_list)):
                    file.write(str(atom_list[j]))
                    for k in range(3):
                        file.write(' '+str("{0:.6f}".format(coordinates[j,k])))
                    file.write("\n")
                file.write("\n")
    return


In [ ]:
def make_input_folder(file, header_file, atoms):
    """
    This function generates directories with the possible combinations of 
    vibrations and generates input files for each directory given a geometry
    optimization log file, a header file and the number of atoms in the molecule
    """

    cwd = os.getcwd()
    for i in range(3*atoms-6):
        vib1=0
        vib2=0
        for j in range(3*atoms-6):
            vib1=i+1
            vib2=j+1
            if vib2 > vib1:
                os.mkdir('v'+str(vib1)+'v'+str(vib2))
                path=str(cwd)+'/'+'v'+str(vib1)+'v'+str(vib2)
                os.chdir(path)
                freqs,Q_mat_list=extract_freq(file,atoms)
                initial_coordinates,atom_list=extract_initial_coordinates(file,atoms)
                new_coords=new_coordinates(initial_coordinates,Q_mat_list[int(vib1)-1],Q_mat_list[int(vib2)-1],0.001,10,atoms)
                input_files(new_coords,atom_list,'vib'+str(vib1)+'-vib'+str(vib2)+'-',header_file,10)
                os.chdir('..')

    return


In [ ]:
file = sys.argv[1] #log file with initial structure and vibrational freqs
header_file = sys.argv[2] #header file with route and lin0 section

atoms= int( sys.argv[3]) #number of atoms in the molecule

In [ ]:
make_input_folder(file, header_file,atoms)